In [1]:
import sys, re
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly.graph_objs import *
init_notebook_mode(connected=True)
import string
import glob
import os

In [2]:
train = (open('Data/train.txt').read()).split('\n')
dev = (open('Data/dev.txt').read()).split('\n')

In [3]:
def tokenize(data):
    index = 0;
    ls = ['@','@']
    new1 = []
    for i in data:
        if i != '.':
            ls.append(i)
        else:
            ls.append('.')
            new1.append(ls)
            ls = ['@', '@']
    i = 0
    for sentence in new1:
        sentence = list(filter(None, sentence))
        new1[i] = sentence
        i=i+1 
    return new1

In [4]:
train = tokenize(train)
dev = tokenize(dev)

In [5]:
def language_model(n, corpus):
    my_dict = dict()
    
    for sentence in corpus:
        for i in range(0, len(sentence) - (n - 1)):
            if n == 1:
                key = sentence[i]
            else:
                key = tuple(sentence[i : i + n])   
            if key in my_dict:
                my_dict[key] += 1
            else:
                my_dict[key] = 1
    return my_dict
train_grams = language_model(3, train)
train_uni_grams = language_model(1, train)

In [6]:
keyDict = ["Vt", "Vm", "V0", "Vform", "SVA", "ArtOrDet", "Nn", "Npos", "Pform", "Pref", "Prep", "Wci", "Wa", 
       "Wform", "Wtone", "Srun", "Smod", "Spar", "Sfrag", "Ssub", "WOinc", "WOadv", "Trans", "Mec", "Rloc-",
       "Cit", "Others", "Um"]

In [7]:
def fill_classes(corpus):
    classes = dict([(keys, []) for keys in keyDict])
    for keys in corpus:
        if ' ' in keys[2]:
            p = keys[2].split(' ')
            p = list(filter(None, p))
            classes[p[-1]].append(keys)
    return classes

def fill_classes_uni(corpus):
    classes = dict([(keys, []) for keys in keyDict])
    for keys in corpus:
        if ' ' in keys:
            p = keys.split(' ')
            p = list(filter(None, p))
            classes[p[-1]].append(keys)
    return classes

In [8]:
classes = fill_classes(train_grams)
uni_classes = fill_classes_uni(train_uni_grams)

In [9]:
def total_words_class(cls, model, corpus):
    sum_words = 0
    for i in model[cls]:
        sum_words += corpus[i]
    return sum_words

In [10]:
def seq_prob(seq):
    count1 = 0
    count2 = 0
    count3 = 0
    for i in train_grams:
        p1 = list(filter(None, i[0].split(' ')))
        p2 = list(filter(None, i[1].split(' ')))
        p3 = list(filter(None, i[2].split(' '))) 
        if p1[0] == seq[0] and p2[0] == seq[1] and p3[0] == seq[2]: 
            count1 = count1 + train_grams[i]
        if p2[0] == seq[1] and p3[0] == seq[2]:
            count2 = count2 + train_grams[i]
        if p3[0] == seq[2]:
            count3 = count3 + train_grams[i]
            
    return [count1, count2, count3]

In [11]:
def naive_bayes(seq):
    counts = seq_prob(seq)
    prob_dict = dict()
    for key in classes:
        count = 0
        for grams in classes[key]:
            p1 = list(filter(None, grams[0].split(' ')))
            p2 = list(filter(None, grams[1].split(' ')))
            p3 = list(filter(None, grams[2].split(' ')))
            if counts[0] != 0:
                if p1[0] == seq[0] and p2[0] == seq[1] and p3[0] == seq[2]: 
                    count = count + train_grams[grams]
            elif counts[1] != 0:
                if p2[0] == seq[1] and p3[0] == seq[2]:
                    count = count + train_grams[grams]
            elif p3[0] == seq[2]:
                count = count + train_grams[grams]      
        
        if counts[0] != 0:
            prob_s_c = (count / total_words_class(key, classes, train_grams))
        elif counts[1] != 0:
            prob_s_c = 0.08 * (count / total_words_class(key, classes, train_grams))
        else:
            prob_s_c = 0.008 * (count / total_words_class(key, classes, train_grams))
        
        prob_c = (total_words_class(key, uni_classes, train_uni_grams))/sum(train_uni_grams.values())
        prob_dict[key] = prob_s_c
    return prob_dict

In [12]:
counts_in_clss = dict()
for key in classes:
    counts_in_clss[key] = total_words_class(key, uni_classes, train_uni_grams)

In [13]:
iplot([{"x" : [''.join(x) for x in list(zip(*list(counts_in_clss.items())))[0]], "y": list(zip(*list(counts_in_clss.items())))[1]}])

In [14]:
prob_key_dict = {"Vt":10**-4, "Vm":10**-6, "V0":10**-6, "Vform":10**-4, "SVA":10**-5, 
           "ArtOrDet":10**-3, "Nn":10**-4, "Npos":10**-6, "Pform":10**-6, "Pref":10**-5, 
           "Prep":10**-5, "Wci":10**-4, "Wa":10**-7, "Wform":10**-5, "Wtone":10**-5, 
           "Srun":10**-5, "Smod":10**-7, "Spar":10**-6, "Sfrag":10**-6, "Ssub":10**-6, 
           "WOinc":10**-4, "WOadv":10**-5, "Trans":10**-1, "Mec":10**-5, "Rloc-":10**-5,
       "Cit":10**-5, "Others":10**-5, "Um":10**-4}

In [ ]:
os.remove("dev_results.txt")
f = open("dev_results.txt", "a")
for lst in dev:
    for i in range(0, len(lst) - 2):
        temp = tuple(lst[i : i + 3])
        prob_dict = naive_bayes(temp)
        clss = max(prob_dict, key = prob_dict.get)
        if prob_dict[clss] > prob_key_dict[clss]:
            f.write(temp[2] + '      ' + clss + '\n')
        else:
            f.write(temp[2] + '\n')
f.close()

In [19]:
# for lst in dev[0:3]:
#     for i in range(0, len(lst) - 2):
#         temp = tuple(lst[i : i + 3])
#         prob_dict, counts = naive_bayes(temp)
#         clss = max(prob_dict, key = prob_dict.get)
#         if prob_dict[clss] > prob_key_dict[clss]:
#             print(temp[2], '   ', clss, '   ', correction(clss, temp, classes, counts))
#         else:
#             print(temp[2])

In [17]:
# def correction(cls, seq, grams, counts):
#     selected_key = 0
#     for key in grams[cls]:           
#         p1 = list(filter(None, key[0].split(' ')))
#         p2 = list(filter(None, key[1].split(' ')))
#         p3 = list(filter(None, key[2].split(' ')))
#         if counts[0] != 0:
#             if p1[0] == seq[0] and p2[0] == seq[1] and p3[0] == seq[2]:
#                 if selected_key == 0:
#                     selected_key = key
#                 elif train_grams[key] > train_grams[selected_key]:
#                     selected_key = key
#         elif counts[1] != 0:
#             if p2[0] == seq[1] and p3[0] == seq[2]:
#                 if selected_key == 0:
#                     selected_key = key
#                 elif train_grams[key] > train_grams[selected_key]:
#                     selected_key = key
#         elif p3[0] == seq[2]:
#                 if selected_key == 0:
#                     selected_key = key
#                 elif train_grams[key] > train_grams[selected_key]:
#                     selected_key = key
#     p3 = list(filter(None, selected_key[2].split(' ')))
#     return p3[1]
        